In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
# Import the data 
ltc = pd.read_csv('../data/BC/ngan_merged_OSAtoCOVID.csv')
ltc.info()
ltc.columns.values
# for each in ltc.columns.values:
#     print(each)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Columns: 108 entries, FACILITY_NAME to outbreak
dtypes: float64(24), int64(7), object(77)
memory usage: 250.7+ KB


array(['FACILITY_NAME', 'HCC_CODE', 'HLTH_AUTH', 'OWNERSHIP',
       'STREET_ADDRESS', 'CITY', 'POSTAL', 'PHONE', 'OPERATOR_NAME',
       'OPEN_DATE', 'COUNCIL', 'MEETINGS_HELD', 'PARKING_COSTS',
       'REGULATION', 'ACCRED_STATUS', 'ACCRED_EXPIRY',
       'COMPLAINT_CONTACT', 'COMPLAINT_PHONE', 'SPENDING_ACCOUNT',
       'FOOD_SERVICE', 'FOOD_PREP', 'FOOD_COST_LASTYR',
       'FOOD_COST_CURRENTYR', 'LANGUAGES', 'DCH_NURSE_LASTYR',
       'DCH_ALLIED_LASTYR', 'DCH_TOTAL_LASTYR', 'DCH_NURSE_CURRENTYR',
       'DCH_ALLIED_CURRENTYR', 'DCH_TOTAL_CURRENTYR', 'FEES_CABLE',
       'FEES_TELE', 'FEES_INTERNET', 'FEES_OTHER', 'BEDS_PRIVATE',
       'BEDS_PUBLIC', 'BEDS_TOTAL', 'ROOMS_PRIVATE', 'ROOMS_SEMI',
       'ROOMS_MULTI', 'COMPLAINTS', 'SUB_COMPLAINTS', 'INCIDENT_OUTBREAK',
       'INCIDENT_ABUSE', 'INCIDENT_FALL', 'INCIDENT_POISON',
       'INCIDENT_MEDICATION', 'INCIDENT_WANDERING', 'INCIDENT_INJURY',
       'INCIDENT_AGGRESSION', 'INCIDENT_OUTBREAK_100',
       'INCIDENT_ABUSE_100',

In [3]:
# There are 5 homes where all ROOMS_* preditors == 0
temp = ltc.loc[ltc['ROOMS_PRIVATE'] == 0]
temp[['FACILITY_NAME', 'BEDS_PRIVATE', 'BEDS_TOTAL', 'ROOMS_PRIVATE', 'ROOMS_SEMI', 'ROOMS_MULTI']]

,FACILITY_NAME,BEDS_PRIVATE,BEDS_TOTAL,ROOMS_PRIVATE,ROOMS_SEMI,ROOMS_MULTI
26,Brookhaven Care Centre,0,84,0,0,0
66,Dufferin Care Centre,25,153,0,0,0
101,Glenmore Lodge,18,118,0,0,0
107,Haida Gwaii Hospital and Health Centre,0,8,0,2,2
123,Holyrood Manor,2,125,0,0,0
181,Nicola Lodge,18,230,0,0,0


# Impute where all ROOMS_* predictors == 0

The blank room counts are due to the self-reported nature of the data. In these cases, the facilities did not respond for our currently published Quick Facts Directory. They DID respond, however, to our recent targeted inquiries to resolve room count issues mentioned above.
    - Brookhaven Care Centre – 104 public beds, 96 single rooms, 4 double rooms
    - Dufferin Care Centre – 128 public beds, 25 private beds, 97 single rooms, 28 double rooms
    - Glenmore Lodge – 100 public beds, 18 private beds, 118 single rooms
    - Holyrood Manor – 124 public beds, 1 private bed, 77 single rooms, 24 double rooms
    - Nicola Lodge – 212 public beds, 18 private beds, 220 single rooms, 5 double rooms

In [4]:
# Impute
# Brookhaven Care Centre
ltc.at[26, 'BEDS_TOTAL'] = 104
ltc.at[26, 'ROOMS_PRIVATE'] = 97
ltc.at[26, 'ROOMS_SEMI'] = 28

# ltc.loc[ltc['FACILITY_NAME'] == 'Dufferin Care Centre'] #66
ltc.at[66, 'ROOMS_PRIVATE'] = 96
ltc.at[66, 'ROOMS_SEMI'] = 4

# ltc.loc[ltc['FACILITY_NAME'] == 'Glenmore Lodge'] #101
ltc.at[101, 'ROOMS_PRIVATE'] = 118

# ltc.loc[ltc['FACILITY_NAME'] == 'Holyrood Manor'] #123
ltc.at[123, 'BEDS_TOTAL'] = 125
ltc.at[123, 'BEDS_PRIVATE'] = 1
ltc.at[123, 'ROOMS_PRIVATE'] = 77
ltc.at[123, 'ROOMS_SEMI'] = 24

# ltc.loc[ltc['FACILITY_NAME'] == 'Nicola Lodge'] #181
ltc.at[181, 'ROOMS_PRIVATE'] = 220
ltc.at[181, 'ROOMS_SEMI'] = 5

In [6]:
ls = ['Brookhaven Care Centre',
    'Dufferin Care Centre',
    'Glenmore Lodge',
    'Holyrood Manor',
    'Nicola Lodge'
]

temp = ltc.loc[ltc['FACILITY_NAME'].isin(ls)]
temp[['FACILITY_NAME', 'BEDS_PRIVATE', 'BEDS_TOTAL', 'ROOMS_PRIVATE', 'ROOMS_SEMI', 'ROOMS_MULTI']]

,FACILITY_NAME,BEDS_PRIVATE,BEDS_TOTAL,ROOMS_PRIVATE,ROOMS_SEMI,ROOMS_MULTI
26,Brookhaven Care Centre,0,104,97,28,0
66,Dufferin Care Centre,25,153,96,4,0
101,Glenmore Lodge,18,118,118,0,0
123,Holyrood Manor,1,125,77,24,0
181,Nicola Lodge,18,230,220,5,0


In [10]:
# 248 homes have 0 multi-rooms
# ltc['ROOMS_MULTI'].value_counts()

# 128 homes have 0 semi-rooms
ltc['ROOMS_SEMI'].value_counts()

0     128
4     21 
5     18 
2     14 
6     13 
8     12 
1     10 
3     9  
7     6  
20    6  
9     5  
10    5  
13    5  
12    4  
21    4  
14    4  
16    3  
11    3  
18    3  
36    3  
24    3  
17    2  
22    2  
29    2  
74    1  
49    1  
48    1  
43    1  
37    1  
31    1  
33    1  
78    1  
28    1  
23    1  
19    1  
90    1  
Name: ROOMS_SEMI, dtype: int64

# Create the Complete Dataset ltc_com
# Adjust predictors

In [ ]:
# ltc.dtypes
# ltc['outbreak'].value_counts()

In [11]:
ltc_com = ltc

# # We see that about 2/3 of the homes have 0 private beds
# ltc_com['BEDS_PRIVATE'].value_counts()

# Define proportion of private vs public beds
ltc_com['BEDS_PRIVATEprop'] = ltc_com['BEDS_PRIVATE']/ltc_com['BEDS_TOTAL']

# Define a binary variable for private beds
conditions = [ltc_com['BEDS_PRIVATE'] == 0]
outputs = [0]
ltc_com['BEDS_PRIVATEbinary'] = np.select(conditions, outputs, 1)

# Define a binary variable for rooms_shared
conditions = [ltc_com['ROOMS_SEMI'].eq(0) & ltc_com['ROOMS_MULTI'].eq(0)]
outputs = [0]
ltc_com['rooms_shared'] = np.select(conditions, outputs, 1)

# Define binary variables for presence of semi or multi rooms
conditions = [ltc_com['ROOMS_SEMI'] == 0]
outputs = [0]
ltc_com['ROOMS_SEMIbinary'] = np.select(conditions, outputs, 1)

conditions = [ltc_com['ROOMS_MULTI'] == 0]
outputs = [0]
ltc_com['ROOMS_MULTIbinary'] = np.select(conditions, outputs, 1)

# Define new variables for resident and family councils
conditions = [ltc_com['COUNCIL'].str.contains('Resident', regex=False, na=False)]
outputs = [1]
ltc_com['RESIDENT_COUNCIL'] = np.select(conditions, outputs, 0)

conditions = [ltc_com['COUNCIL'].str.contains('Family', regex=False, na=False)]
outputs = [1]
ltc_com['FAMILY_COUNCIL'] = np.select(conditions, outputs, 0)

# Define new variables for REGULATION
conditions = [ltc_com['REGULATION'].str.contains('Hospital', regex=False, na=False)]
outputs = [1]
ltc_com['hospital_act'] = np.select(conditions, outputs, 0)

conditions = [ltc_com['REGULATION'].str.contains('Community', regex=False, na=False)]
outputs = [1]
ltc_com['community_act'] = np.select(conditions, outputs, 0)


In [12]:
ltc_com.head()

,FACILITY_NAME,HCC_CODE,HLTH_AUTH,OWNERSHIP,STREET_ADDRESS,CITY,POSTAL,PHONE,OPERATOR_NAME,OPEN_DATE,COUNCIL,MEETINGS_HELD,PARKING_COSTS,REGULATION,ACCRED_STATUS,ACCRED_EXPIRY,COMPLAINT_CONTACT,COMPLAINT_PHONE,SPENDING_ACCOUNT,FOOD_SERVICE,FOOD_PREP,FOOD_COST_LASTYR,FOOD_COST_CURRENTYR,LANGUAGES,DCH_NURSE_LASTYR,DCH_ALLIED_LASTYR,DCH_TOTAL_LASTYR,DCH_NURSE_CURRENTYR,DCH_ALLIED_CURRENTYR,DCH_TOTAL_CURRENTYR,FEES_CABLE,FEES_TELE,FEES_INTERNET,FEES_OTHER,BEDS_PRIVATE,BEDS_PUBLIC,BEDS_TOTAL,ROOMS_PRIVATE,ROOMS_SEMI,ROOMS_MULTI,COMPLAINTS,SUB_COMPLAINTS,INCIDENT_OUTBREAK,INCIDENT_ABUSE,INCIDENT_FALL,INCIDENT_POISON,INCIDENT_MEDICATION,INCIDENT_WANDERING,INCIDENT_INJURY,INCIDENT_AGGRESSION,INCIDENT_OUTBREAK_100,INCIDENT_ABUSE_100,INCIDENT_FALL_100,INCIDENT_POISON_100,INCIDENT_MEDICATION_100,INCIDENT_WANDERING_100,INCIDENT_INJURY_100,INCIDENT_AGGRESSION_100,ADVERSE_EVENT,AGE,FEMALE,AGE_85_PLUS,AGE_UNDER_65,STAY_LENGTH,DEPRESSION,ADL_DEPENDENT,CPS_SEVERE,DEMENTIA,ABS_PHYS_ABUSIVE,ISE,ISE_LOW,CMI,THERAPY_PT,THERAPY_RT,THERAPY_OT,MEDS_DEPRESSION,MEDS_ANTIPSYCHOTICS,RESTRAINTS,WEBPAGE,SURVEY_URL,MISSING_SURVEY,CONTRACTED_NURSING,CONTRACTED_CARE_AIDES,CONTRACTED_HOUSEKEEPING,CONTRACTED_PLANT,PER_DIEM_LASTYR,PER_DIEM_CURRENTYR,INSPECTIONS,INFRACTIONS,INFRACTIONS_LICENSING,INFRACTIONS_FACILITY,INFRACTIONS_STAFFING,INFRACTIONS_POLICY,INFRACTIONS_CARE,INFRACTIONS_DISEASE,INFRACTIONS_FOOD,INFRACTIONS_MEDICATION,INFRACTIONS_PROGRAM,INFRACTIONS_REPORTING,INFRACTIONS_RIGHTS,INFRACTIONS_OTHER,latitude,longitude,cleaned_name,Facility Name,Total Confirmed Cases,Total Deaths,outbreak,BEDS_PRIVATEprop,BEDS_PRIVATEbinary,rooms_shared,ROOMS_SEMIbinary,ROOMS_MULTIbinary,RESIDENT_COUNCIL,FAMILY_COUNCIL,hospital_act,community_act
0,Aberdeen Hospital,9A616,Vancouver Island Health,HA,1450 Hillside Ave.,Victoria,V8T2B7,(250) 370-5648,Vancouver Island Health,1982,Resident & Family Council (combined),Regular Schedule,Yes (fee charged),Hospital Act,Accredited,April 2022,Manager,(250) 370-5678,Yes,Operator & Outside Contractor,Yes,$8.06,$8.16,English,4.36,0.38,4.74,4.36,0.38,4.74,No,No,No,NaN,0,100,100,19,4,25,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,69,51.6%,21.7%,43.3%,844,24.3%,45.6%,16.0%,6.8%,0.5%,4.0,16.0%,0.77,9.2%,51.5%,45.1%,48.1%,22.7%,3.9%,https://www.islandhealth.ca/our-locations/long-term-care-locations/aberdeen-hospital,https://surveybcseniors.org/carehomeresults/aberdeen-hospital-island/,NaN,No,No,Yes,No,NaN,NaN,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.442197,-123.343242,aberdeen hospital,NaN,NaN,NaN,0,0.000000,0,1,1,1,1,1,1,0
1,Acacia Ty Mawr,99549,Vancouver Island Health,PFP,2655 E Shawnigan Lake Rd,Shawnigan Lake,V0R2W0,(250) 743-2124,Trillium Communities,1990,Family Council,Regular Schedule,Yes (no fee),Community Care & Assisted Living Act,Accredited,2020,Director of Care,(250) 743-2124 ext 2,Yes,Operator,Yes,$10.92,$12.35,"English, German, Indonesian, Dutch",2.70,0.11,2.80,2.91,0.15,3.06,No,No,NaN,Hygiene items,0,35,35,31,2,0,1,0,0,0,2,0,0,0,1,3,0.0,0.0,5.7,0.0,0.0,0.0,2.9,8.6,not applicable,84,69.8%,52.2%,2.5%,862,10.4%,30.6%,38.9%,95.1%,10.4%,2.0,65.3%,0.56,0.0%,0.0%,0.0%,54.2%,37.8%,2.0%,https://www.trilliumcommunities.com/location/acacia-ty-mawr/,https://surveybcseniors.org/carehomeresults/acacia-ty-mawr-island/,NaN,No,No,No,Some,$230.96,$239.10,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,48.649871,-123.623580,acacia ty mawr,NaN,NaN,NaN,0,0.000000,0,1,1,0,0,1,0,1
2,Acropolis Manor,9A8ID,Northern Health,HA,1325 Summit Avenue,Prince Rupert,V8J4C1,(250) 622-6400,Northern Health,2009,"Resident Council, Family Council",Regular Schedule,Yes (no fee),Community Care & Assisted Living Act,Accredited,June 2022,Site Manager,(250) 622-6450,Yes,Operator,No,$13.45,$14.09,"Filipino, Portuguese, Italian, Punjabi, Hindi, Tsimshianic, Cebuano, English",3.14,0.32,3.47,3.08,0.35,3.43,No,No,No,NaN,0,61,61,49,6,0,1,1,1,1,8,0,0,3,1,3,1.6,1.6,13.1,0.0,0.0,4.9,1.6,4.9,not applicable,83,59.2%,50.7%,4.4%,2,1

In [13]:
# Drop irrelevant columns
ls = [
    'PHONE', 
    'OPERATOR_NAME', # there are 100 unique values
    'OPEN_DATE',
    'COUNCIL',
    'MEETINGS_HELD', 
    'PARKING_COSTS',
    'ACCRED_EXPIRY',
    'COMPLAINT_CONTACT', 
    'COMPLAINT_PHONE', 
    'SPENDING_ACCOUNT',
    'FOOD_SERVICE', 
    'FOOD_PREP', 
    'FOOD_COST_LASTYR',
    'FOOD_COST_CURRENTYR', 
    'LANGUAGES', 
    'FEES_CABLE',
    'FEES_TELE', 
    'FEES_INTERNET', 
    'FEES_OTHER', 
    'BEDS_PRIVATE',
    'BEDS_PUBLIC',
    'ADVERSE_EVENT',    
    'WEBPAGE', 
    'SURVEY_URL',
    'MISSING_SURVEY',
    'CONTRACTED_NURSING',
    'CONTRACTED_CARE_AIDES',
    'CONTRACTED_HOUSEKEEPING',
    'CONTRACTED_PLANT',
    'PER_DIEM_LASTYR',
    'PER_DIEM_CURRENTYR',
    'cleaned_name', 
    'Facility Name'
]

ltc_com = ltc_com.drop(ls, axis = 1)
ltc_com.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 84 columns):
FACILITY_NAME              297 non-null object
HCC_CODE                   297 non-null object
HLTH_AUTH                  297 non-null object
OWNERSHIP                  297 non-null object
STREET_ADDRESS             297 non-null object
CITY                       297 non-null object
POSTAL                     297 non-null object
REGULATION                 297 non-null object
ACCRED_STATUS              297 non-null object
DCH_NURSE_LASTYR           293 non-null float64
DCH_ALLIED_LASTYR          293 non-null float64
DCH_TOTAL_LASTYR           293 non-null float64
DCH_NURSE_CURRENTYR        294 non-null float64
DCH_ALLIED_CURRENTYR       294 non-null float64
DCH_TOTAL_CURRENTYR        294 non-null float64
BEDS_TOTAL                 297 non-null int64
ROOMS_PRIVATE              297 non-null int64
ROOMS_SEMI                 297 non-null int64
ROOMS_MULTI                297 non-null int64


In [14]:
# Export the complete dataset
ltc_com.to_csv(r'../data/BC/bc_ltc_complete_sep18.csv', index = False)

# Create selected variables dataset ltc_sel

In [15]:
ltc_sel = ltc_com
ltc_sel = ltc_sel.dropna(axis=1)
ltc_sel.info()
# ltc_sel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 25 columns):
FACILITY_NAME         297 non-null object
HCC_CODE              297 non-null object
HLTH_AUTH             297 non-null object
OWNERSHIP             297 non-null object
STREET_ADDRESS        297 non-null object
CITY                  297 non-null object
POSTAL                297 non-null object
REGULATION            297 non-null object
ACCRED_STATUS         297 non-null object
BEDS_TOTAL            297 non-null int64
ROOMS_PRIVATE         297 non-null int64
ROOMS_SEMI            297 non-null int64
ROOMS_MULTI           297 non-null int64
latitude              297 non-null float64
longitude             297 non-null float64
outbreak              297 non-null int64
BEDS_PRIVATEprop      297 non-null float64
BEDS_PRIVATEbinary    297 non-null int64
rooms_shared          297 non-null int64
ROOMS_SEMIbinary      297 non-null int64
ROOMS_MULTIbinary     297 non-null int64
RESIDENT_COUNCIL   

In [16]:
# Reorder columns in df
ltc_sel = ltc_sel[[
        'FACILITY_NAME',
        'HCC_CODE',
        'HLTH_AUTH',
        'OWNERSHIP',
        'STREET_ADDRESS',
        'CITY',
        'POSTAL',
        'REGULATION',
        'hospital_act',
        'community_act',
        'ACCRED_STATUS',
        'RESIDENT_COUNCIL',
        'FAMILY_COUNCIL',
        'BEDS_TOTAL',
        'BEDS_PRIVATEprop',
        'BEDS_PRIVATEbinary',
        'ROOMS_PRIVATE',
        'ROOMS_SEMI',
        'ROOMS_MULTI',
        'ROOMS_SEMIbinary',
        'ROOMS_MULTIbinary',
        'rooms_shared',
        'latitude',
        'longitude',
        'outbreak',
]]
ltc_sel.info()
ltc_sel.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 25 columns):
FACILITY_NAME         297 non-null object
HCC_CODE              297 non-null object
HLTH_AUTH             297 non-null object
OWNERSHIP             297 non-null object
STREET_ADDRESS        297 non-null object
CITY                  297 non-null object
POSTAL                297 non-null object
REGULATION            297 non-null object
hospital_act          297 non-null int64
community_act         297 non-null int64
ACCRED_STATUS         297 non-null object
RESIDENT_COUNCIL      297 non-null int64
FAMILY_COUNCIL        297 non-null int64
BEDS_TOTAL            297 non-null int64
BEDS_PRIVATEprop      297 non-null float64
BEDS_PRIVATEbinary    297 non-null int64
ROOMS_PRIVATE         297 non-null int64
ROOMS_SEMI            297 non-null int64
ROOMS_MULTI           297 non-null int64
ROOMS_SEMIbinary      297 non-null int64
ROOMS_MULTIbinary     297 non-null int64
rooms_shared          2

,FACILITY_NAME,HCC_CODE,HLTH_AUTH,OWNERSHIP,STREET_ADDRESS,CITY,POSTAL,REGULATION,hospital_act,community_act,ACCRED_STATUS,RESIDENT_COUNCIL,FAMILY_COUNCIL,BEDS_TOTAL,BEDS_PRIVATEprop,BEDS_PRIVATEbinary,ROOMS_PRIVATE,ROOMS_SEMI,ROOMS_MULTI,ROOMS_SEMIbinary,ROOMS_MULTIbinary,rooms_shared,latitude,longitude,outbreak
0,Aberdeen Hospital,9A616,Vancouver Island Health,HA,1450 Hillside Ave.,Victoria,V8T2B7,Hospital Act,1,0,Accredited,1,1,100,0.000000,0,19,4,25,1,1,1,48.442197,-123.343242,0
1,Acacia Ty Mawr,99549,Vancouver Island Health,PFP,2655 E Shawnigan Lake Rd,Shawnigan Lake,V0R2W0,Community Care & Assisted Living Act,0,1,Accredited,0,1,35,0.000000,0,31,2,0,1,0,1,48.649871,-123.623580,0
2,Acropolis Manor,9A8ID,Northern Health,HA,1325 Summit Avenue,Prince Rupert,V8J4C1,Community Care & Assisted Living Act,0,1,Accredited,1,1,61,0.000000,0,49,6,0,1,0,1,54.304055,-130.330000,0
3,Adanac Park Lodge,919G3,Vancouver Coastal Health,PNP,851 Boundary Road,Vancouver,V5K4T2,Community Care & Assisted Living Act,0,1,Not Accredited,1,0,73,0.000000,0,67,3,0,1,0,1,49.277077,-123.024596,0
4,Agassiz Seniors Community,936E0,Fraser Health,PFP,1525 MacKay Crescent,Agassiz,V0M1A3,Community Care & Assisted Living Act,0,1,Unknown,1,1,74,0.148649,1,68,0,0,0,0,0,49.233502,-121.768161,0


In [17]:
# Export the selected dataset
ltc_sel.to_csv(r'../data/BC/bc_ltc_selected_sep18.csv', index = False)